Each video was separated into frames. The face_recognition library was used to detect faces in each frame (if any). Upon detecting a face, the coordinates of the face + some padding was used to extract the face from the frame as a new image. These images were both stored in a folder named "Real" or "Fake" based on the type of video from which it was extracted, as well as, a pandas dataframe was built to hold the name of each image and the associated label. Some percentage of videos are used for training a classifier, while the remainder is used to validate the performance of the classifier. This distinction is made via a change to the path where the images are stored. 

In [3]:
import numpy as np
import pandas as pd
import cv2 as cv
import os
import pandas as pd
from IPython.display import Video
from IPython.display import HTML
from PIL import Image

import face_recognition

In [4]:
real_dir = 'Real/'
real_video_files = [real_dir + x for x in os.listdir(real_dir)]

fake_dir = 'Fake/'
fake_video_files = [fake_dir + x for x in os.listdir(fake_dir)]

#It may be worth extracting some padding around the face
padding = 40

image_num = 100000
data = []
path = 'Images/Train/Real'

train_percentage = 0.5
#Extract images of the faces in the real videos and pair the name of each image 
#with the word "Real"
for vid_index in range(len(real_video_files)):
    if vid_index> len(real_video_files)*train_percentage:
        path = 'Images/Val/Real'
    video_file = real_video_files[vid_index]
    cap = cv.VideoCapture(video_file)
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            frames.append(frame)
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    #Extract every 20th frame
    frames = frames[0::20]
    for frame in frames:
        face_locations = face_recognition.face_locations(frame)
    
        #If no face is found in the frame, move on.
        if len(face_locations) == 0:
            continue

        top, right, bottom, left = face_locations[0]
        frame_face = frame[top-padding:bottom+padding, left-padding:right+padding]
        image = cv.cvtColor(frame_face, cv.COLOR_BGR2RGB)
        image_name = '{}.jpg'.format(image_num)
        image_num += 1
        #cv.imwrite(image_name,image)
        cv.imwrite(os.path.join(path , image_name),image)
        data.append([image_name,'Real'])
    

#Extract images of the faces in the fake videos and pair the name of each image 
#with the word "Fake"
path = 'Images/Train/Fake'
for vid_index in range(len(fake_video_files)):
    if vid_index > len(fake_video_files)*train_percentage:
        path = 'Images/Val/Fake'
    video_file = fake_video_files[vid_index]
    cap = cv.VideoCapture(video_file)
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            frames.append(frame)
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    #Extract every 20th frame
    frames = frames[0::20]
    for frame in frames:
        face_locations = face_recognition.face_locations(frame)
    
        #If no face is found in the frame, move on.
        if len(face_locations) == 0:
            continue

        top, right, bottom, left = face_locations[0]
        frame_face = frame[top-padding:bottom+padding, left-padding:right+padding]
        image = cv.cvtColor(frame_face, cv.COLOR_BGR2RGB)
        image_name = '{}.jpg'.format(image_num)
        image_num += 1
        #cv.imwrite(image_name,image)
        cv.imwrite(os.path.join(path , image_name),image)
        data.append([image_name,'Fake'])
    

#Build a pandas dataframe with the image name and the type of image (real/fake)
#This will not be used in the current iteration but may be useful for further experiments
dataset = pd.DataFrame(data,columns=['Image','Type'])    
dataset.to_csv("full_dataset2.csv", sep=',',index=False)

'''
Alternately, images can be extracted to one directory and 
the image name + label can be stored in a csv as shown in the comments below.
'''

In [6]:
'''real_dir = 'Real/'
real_video_files = [real_dir + x for x in os.listdir(real_dir)]

fake_dir = 'Fake/'
fake_video_files = [fake_dir + x for x in os.listdir(fake_dir)]

#It may be worth extracting some padding around the face
padding = 0

image_num = 0
data = []
path = 'Images/'

val_split = 0.25
#Extract images of the faces in the real videos and pair the name of each image 
#with the word "Real"
for video_file in real_video_files:
    cap = cv.VideoCapture(video_file)
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            frames.append(frame)
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    for frame in frames:
        face_locations = face_recognition.face_locations(frame)
    
        #If no face is found in the frame, move on.
        if len(face_locations) == 0:
            continue

        top, right, bottom, left = face_locations[0]
        frame_face = frame[top-padding:bottom+padding, left-padding:right+padding]
        image = cv.cvtColor(frame_face, cv.COLOR_BGR2RGB)
        image_name = '{}.jpg'.format(image_num)
        image_num += 1
        #cv.imwrite(image_name,image)
        cv.imwrite(os.path.join(path , image_name),image)
        data.append([image_name,'Real'])
    

#Extract images of the faces in the fake videos and pair the name of each image 
#with the word "Fake"
for video_file in fake_video_files:
    cap = cv.VideoCapture(video_file)
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            frames.append(frame)
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    for frame in frames:
        face_locations = face_recognition.face_locations(frame)
    
        #If no face is found in the frame, move on.
        if len(face_locations) == 0:
            continue

        top, right, bottom, left = face_locations[0]
        frame_face = frame[top-padding:bottom+padding, left-padding:right+padding]
        image = cv.cvtColor(frame_face, cv.COLOR_BGR2RGB)
        image_name = '{}.jpg'.format(image_num)
        image_num += 1
        #cv.imwrite(image_name,image)
        cv.imwrite(os.path.join(path , image_name),image)
        data.append([image_name,'Fake'])
    

#Build a pandas dataframe with the image name and the type of image (real/fake)
dataset = pd.DataFrame(data,columns=['Image','Type'])   
dataset.to_csv("full_dataset.csv", sep=',',index=False)